# Análisis Exploratorio de Datos (EDA): 

## Objetivo
Este notebook contiene el análisis exploratorio completo del dataset, siguiendo una estructura organizada y metódica para entender los datos, limpiarlos y prepararlos para modelado.

## Contenido
1. Configuración Inicial y librearías
2. Carga y Vista General de los Datos
3. Limpieza y Preprocesamiento
4. Análisis Exploratorio de los Datos
5. Análisis estadístico
6. Análisis Univariante
7. Análisis Bivariante
8. Conclusiones

Esto es una prueba 

In [3]:
import pandas as pd 
import os

In [2]:
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', None) 

# pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)     # Show all rows
# pd.set_option('display.max_colwidth', None) # Show entire content of each column
pd.set_option('display.width', None)        # No limit on display width
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

In [4]:
# Setting paths
current_dir = os.getcwd()  # Use os.getcwd() to get the current working directory
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, "data")
file_path = os.path.join(data_dir, "")

cars_csv_path = os.path.join(data_dir, "train.csv")

#----------------------------------------------------------------------------------------------------
# Creading dataframes
df_car = pd.read_csv(cars_csv_path)

In [5]:
df_car.sample(5)

,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
119982,119982,Audi,A7 55 Premium,2022,12000,Hybrid,335.0HP 3.0L V6 Cylinder Engine Gasoline/Mild ...,A/T,White,Black,None reported,Yes,75000
84843,84843,Ford,F-250 XL,2016,29719,E85 Flex Fuel,385.0HP 6.2L 8 Cylinder Engine Flex Fuel Capab...,A/T,White,Gray,None reported,Yes,31950
94580,94580,Land,Rover Range Rover Sport HSE,2016,87500,Gasoline,340.0HP 3.0L V6 Cylinder Engine Gasoline Fuel,A/T,Gray,Black,None reported,Yes,28000
58,58,Land,Rover Range Rover Sport HSE,2008,122000,Gasoline,300.0HP 4.4L 8 Cylinder Engine Gasoline Fuel,A/T,White,Beige,None reported,Yes,8000
5889,5889,BMW,M235 i,2013,82000,Gasoline,320.0HP 3.0L Straight 6 Cylinder Engine Gasoli...,6-Speed M/T,Black,Red,None reported,Yes,14999


# 1. Configuración Inicial

Importamos las bibliotecas necesarias y configuramos el entorno de trabajo.

In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import missingno as msno
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
import os


### Configuración del estilo visual

In [32]:

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (6, 4)
%matplotlib inline

In [33]:
print("Librerías importadas y configuración realizada.") 

Librerías importadas y configuración realizada.


### Configuración de pandas

In [34]:
pd.set_option('display.max_rows', 100) 
pd.set_option('display.max_columns', None) 


pd.set_option('display.width', None)        # No limit on display width
pd.set_option('display.expand_frame_repr', False)  # Prevent wrapping

# 2. Carga y Preparación de los Datos

### Cargamos el dataset y realizamos una inspección inicial para entender su estructura.

In [35]:
# Setting paths
current_dir = os.getcwd()  # Use os.getcwd() to get the current working directory
parent_dir = os.path.dirname(current_dir)
data_dir = os.path.join(parent_dir, "data")
file_path = os.path.join(data_dir, "")

car_csv_path = os.path.join(data_dir, "train.csv")
#----------------------------------------------------------------------------------------------------
# Creating dataframes
df_car_price= pd.read_csv(car_csv_path)

df_car_price.head()


,id,brand,model,model_year,milage,fuel_type,engine,transmission,ext_col,int_col,accident,clean_title,price
0,0,MINI,Cooper S Base,2007,213000,Gasoline,172.0HP 1.6L 4 Cylinder Engine Gasoline Fuel,A/T,Yellow,Gray,None reported,Yes,4200
1,1,Lincoln,LS V8,2002,143250,Gasoline,252.0HP 3.9L 8 Cylinder Engine Gasoline Fuel,A/T,Silver,Beige,At least 1 accident or damage reported,Yes,4999
2,2,Chevrolet,Silverado 2500 LT,2002,136731,E85 Flex Fuel,320.0HP 5.3L 8 Cylinder Engine Flex Fuel Capab...,A/T,Blue,Gray,None reported,Yes,13900
3,3,Genesis,G90 5.0 Ultimate,2017,19500,Gasoline,420.0HP 5.0L 8 Cylinder Engine Gasoline Fuel,Transmission w/Dual Shift Mode,Black,Black,None reported,Yes,45000
4,4,Mercedes-Benz,Metris Base,2021,7388,Gasoline,208.0HP 2.0L 4 Cylinder Engine Gasoline Fuel,7-Speed A/T,Black,Beige,None reported,Yes,97500


### Vista general del dataset

In [36]:
df_car_price.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188533 entries, 0 to 188532
Data columns (total 13 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   id            188533 non-null  int64 
 1   brand         188533 non-null  object
 2   model         188533 non-null  object
 3   model_year    188533 non-null  int64 
 4   milage        188533 non-null  int64 
 5   fuel_type     183450 non-null  object
 6   engine        188533 non-null  object
 7   transmission  188533 non-null  object
 8   ext_col       188533 non-null  object
 9   int_col       188533 non-null  object
 10  accident      186081 non-null  object
 11  clean_title   167114 non-null  object
 12  price         188533 non-null  int64 
dtypes: int64(4), object(9)
memory usage: 18.7+ MB


# 3. Limpieza y Preprocesamiento

### Limpieza de Nombres de Columnas y Valores Faltantes

### Detección de duplicados

### Búsqueda de valores desconocidos

## Análisis de valores faltantes

En esta sección, analizamos los valores faltantes en el dataset para entender su distribución y decidir cómo manejarlos.

#### 1.Porcentaje de valores NaN por columna 
El porcentaje de valores `NaN` por columna nos ayuda a identificar qué variables tienen datos faltantes y en qué proporción. Esto es útil para decidir si debemos eliminar columnas, imputar valores o manejar los datos faltantes de otra manera.

### Porcentaje de NaN por país y por región

### Visualización de valores faltantes

En esta sección, utilizamos la biblioteca `missingno` para analizar los valores faltantes en el dataset. Estas visualizaciones nos ayudan a identificar patrones de valores faltantes y su relación con otras columnas.

#### 1. Matriz de valores faltantes
La matriz de valores faltantes muestra la presencia o ausencia de datos en cada columna. Esto es útil para identificar patrones de valores faltantes y su distribución en el dataset.

#### 2. Mapa de calor de correlación de nulidad
El mapa de calor de correlación de nulidad muestra la relación entre las columnas en términos de valores faltantes. Si dos columnas tienen una alta correlación de nulidad, significa que tienden a tener valores faltantes en los mismos registros.

### Análisis de valores faltantes en `Gov_Tax` por año

En esta sección, analizamos la frecuencia de valores faltantes (`NaN`) en la columna `Gov_Tax`, agrupados por año. Esto nos ayuda a identificar si los valores faltantes están concentrados en ciertos períodos de tiempo.

#### Análisis de NaN en 'Education_Campaign' por año

### Análisis de valores faltantes

### Imputación de valores faltantes

Justificación: 

### Eliminación de columnas redundantes. 
Investigar

#### Verificar la Redundancia con la Columna

#### Guardamos el nuevo csv

# 4 Análisis exploratorio de datos

### Distribución 

### Mapa de calor de correlaciones

En este análisis, generamos un mapa de calor para visualizar las correlaciones entre las variables numéricas del dataset.

### Relación entre variables numéricas

En este análisis, generamos un gráfico de pares para visualizar la relación entre todas las variables numéricas del dataset. Este gráfico nos permite identificar patrones, tendencias y posibles correlaciones entre las variables.

### Tendencia

### Relación

### Definición de la variable objetivo

Para este análisis, hemos decidido que la variable objetivo será

#### Justificación:
1. Es una variable

#### Tratamiento de la variable objetivo:
- Verificamos si hay valores faltantes en

# 5 Análisis estadístico

 ## Pruebas paramétricas
 ###   pruebas de normalidad

###   Kolmogorov-Simirnov

Hallazgos: 

 ### Shapiro-Wilk

Hallazgos: 

#### Visualización de la distribución: Gráfico Q-Q

### Prueba de Homegeneidad de varianza

Nos hemos decantado por Levene porque la prueba de Bartlett es más sensible a desviaciones de normalidad en los datos.

#### Levene

#### Generamos un boxplot para comparar visualmente las distribuciones de Diabetes_Prevalence entre los países

#### Prueba de ANOVA:
La prueba de ANOVA permitirá evaluar si las medias de diabetes_prevalence son significativamente diferentes entre los países

## 6. Análisis Univariante
### 6.1 Estadísticas Descriptivas
- Presentamos las estadísticas descriptivas básicas para entender las características generales de las variables numéricas en el dataset.

#### 6.2 Visualización de la Distribución


Hallazgos: 

### Conclusión: